## Airline Route Analysis

**Vision:** To develop a data-driven product that identifies and recommends the most promising U.S. domestic round-trip routes for a new airline venture. This product will prioritize profitability, operational efficiency (punctuality), and strategic market entry, enabling informed investment decisions. It will also incorporate a robust data quality monitoring framework.

**Core User:** Airline leadership, route planning strategists, operations management, and data governance teams.

**Key Objectives:**
 1. Identify the 10 busiest round-trip routes (by flight volume).
 2. Identify the 10 most profitable round-trip routes (pre-airplane cost), detailing revenue, cost, and key operational metrics.
 3. Recommend 5 specific round-trip routes for investment, with clear justification.
 4. Calculate the breakeven point (in terms of round-trip flights) for the $90 million airplane cost for each recommended route.
 5. Propose Key Performance Indicators (KPIs) for monitoring the success of the chosen routes.
 6. Implement a system for tracking and visualizing data quality metrics throughout the data processing pipeline.

 **Phases**
1. Data Foundation
    - Data Loading and Initial understanding
    - Data Cleaning and Quality Assurance
    - Data Transformation and Feature Engineering
2. Analytical Insights & Visualization
    - 10 Busiest Round-Trip Routes
    - 10 Most Profitable Round-Trip Routes
    - Route Recommendation
    - Breakeven Analysis
    - Tableau Reporting for Business
3. Reporting & Strategic Outlook
    - Future KPIs to track
    - Future Roadmap

All analytical steps have been supported by functions in the `airline_scripts` package, and data quality has been a consistent focus, with metrics logged via `dq_utils`. The outputs, including data tables, visualizations, and interpretations, are presented throughout this notebook. Data has also been prepared for potential use in Tableau dashboards.

## Project Setup and Library Imports

In [1]:
# %pip install ydata-profiling ipywidgets

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
# Ensure ydata-profiling is installed if you uncomment the profiling sections
from ydata_profiling import ProfileReport

# Import custom airline scripts
import sys
import os

module_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Now import from airline_scripts
from airline_scripts import config
from airline_scripts import load_utils
from airline_scripts import clean_utils
from airline_scripts import feature_engineering_utils
from airline_scripts import analysis_utils
from airline_scripts import dq_utils

# Configure pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("Libraries imported and project setup configured.")

/Users/Rajz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


airline_scripts package initialized.
config.py loaded with Coordinate column definitions and updated metadata.
Libraries imported and project setup configured.


## Phase 1: Data Foundation
This phase focuses on loading the data, performing initial understanding, cleaning the data, ensuring quality, and then transforming it for analysis. Data quality metrics will be logged throughout this process.

### Task 1.1: Data Loading & Initial Understanding

 - Load Flights, Tickets, and Airport Codes datasets using `load_utils.py`.
 - Log initial DQ metrics for each loaded dataset.
 - Perform initial data profiling (e.g., .head(), .info(), ydata-profiling).
 - Document initial hypotheses and observations.

In [3]:
# Initialize DQ Log for the run
# This creates a new RunID and resets the in-memory DQ log DataFrame.
dq_utils.initialize_dq_log()
print(f"DQ Log initialized for Run ID: {dq_utils.current_run_id}")

# %%
# Load datasets using load_utils
# These functions will use paths from config.py and log initial DQ metrics.
print("Loading datasets...")
flights_raw_df, tickets_raw_df, airport_codes_raw_df = load_utils.load_all_data()

# Check if data loading was successful by inspecting the shapes or if they are None
if flights_raw_df is not None:
    print(f"Flights_raw_df loaded with shape: {flights_raw_df.shape}")
else:
    print("Failed to load Flights_raw_df. Check config.FLIGHTS_FILE path and file existence.")

if tickets_raw_df is not None:
    print(f"Tickets_raw_df loaded with shape: {tickets_raw_df.shape}")
else:
    print("Failed to load Tickets_raw_df. Check config.TICKETS_FILE path and file existence.")

if airport_codes_raw_df is not None:
    print(f"Airport_codes_raw_df loaded with shape: {airport_codes_raw_df.shape}")
else:
    print("Failed to load Airport_codes_raw_df. Check config.AIRPORT_CODES_FILE path and file existence.")

Initializing DQ Log for Run ID: 49ec5f94-94e7-4896-94ba-67fc53ec3155
DQ Log initialized. Ready to log metrics for Run ID: 49ec5f94-94e7-4896-94ba-67fc53ec3155.
DQ Log initialized for Run ID: 49ec5f94-94e7-4896-94ba-67fc53ec3155
Loading datasets...
--- Starting Data Loading Phase (Loading ALL columns for profiling) ---


/Users/Rajz/Documents/Job Search/Capital One/airline_data_challenge_submission/airline_scripts/load_utils.py:21: DtypeWarning: Columns (3,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully loaded all columns for Flights_Raw_Full from /Users/Rajz/Documents/Job Search/Capital One/airline_data_challenge_submission/data/Flights.csv. Shape: (1915886, 16)
Successfully loaded all columns for Tickets_Raw_Full from /Users/Rajz/Documents/Job Search/Capital One/airline_data_challenge_submission/data/Tickets.csv. Shape: (1167285, 12)
Successfully loaded all columns for Airport_Codes_Raw_Full from /Users/Rajz/Documents/Job Search/Capital One/airline_data_challenge_submission/data/Airport_Codes.csv. Shape: (55369, 8)
--- Data Loading Phase Complete (All columns loaded) ---
Flights_raw_df loaded with shape: (1915886, 16)
Tickets_raw_df loaded with shape: (1167285, 12)
Airport_codes_raw_df loaded with shape: (55369, 8)


##### Flights Data - Profiling

In [4]:
# Profiling Flights Data
if flights_raw_df is not None:
    print("\n--- Flights Raw Data ---")
    print("Head:")
    print(flights_raw_df.head())
    print("\nInfo:")
    flights_raw_df.info()
    print("\nDescribe:")
    print(flights_raw_df.describe(include='all'))
    # Generate ydata-profiling report (optional, can be time-consuming for large datasets)
    # try:
    #     print(f"Generating profiling report for Flights data, saving to {config.PROFILING_REPORT_FLIGHTS}")
    #     profile_flights = ProfileReport(flights_raw_df, title="Flights Data Profiling Report")
    #     profile_flights.to_file(config.PROFILING_REPORT_FLIGHTS)
    #     print(f"Flights profiling report saved to {config.PROFILING_REPORT_FLIGHTS}")
    # except Exception as e:
    #     print(f"Could not generate ydata-profiling report for flights: {e}")
    #     print("Ensure 'ydata-profiling' is installed and the data directory/file paths in config.py are correct.")
else:
    print("Flights data (flights_raw_df) is None, was not loaded correctly. Skipping profiling.")


--- Flights Raw Data ---
Head:
      FL_DATE OP_CARRIER TAIL_NUM OP_CARRIER_FL_NUM  ORIGIN_AIRPORT_ID ORIGIN  \
0  2019-03-02         WN   N955WN              4591              14635    RSW   
1  2019-03-02         WN   N8686A              3231              14635    RSW   
2  2019-03-02         WN   N201LV              3383              14635    RSW   
3  2019-03-02         WN   N413WN              5498              14635    RSW   
4  2019-03-02         WN   N7832A              6933              14635    RSW   

  ORIGIN_CITY_NAME  DEST_AIRPORT_ID DESTINATION DEST_CITY_NAME  DEP_DELAY  \
0   Fort Myers, FL            11042         CLE  Cleveland, OH      -8.00   
1   Fort Myers, FL            11066         CMH   Columbus, OH       1.00   
2   Fort Myers, FL            11066         CMH   Columbus, OH       0.00   
3   Fort Myers, FL            11066         CMH   Columbus, OH      11.00   
4   Fort Myers, FL            11259         DAL     Dallas, TX       0.00   

   ARR_DELAY  CANC

Initial Hypotheses or Observations:
 * Flights Data:
     * `FL_DATE` should be converted to datetime format. Has only Q1 2019 Data
     * `CANCELLED` - Convert to 'int' as is likely a binary indicator (0 or 1).
     * `DISTANCE`, `AIR_TIME`, - Convert to Float
     *  `DEP_DELAY`, `ARR_DELAY`, `AIR_TIME`, has close to 2.5% to 3% Missing values and outliers
     *  `DISTANCE` Columns have values like 'Twenty', 'Hundred' , '****', ' ' etc. which needs to be cleaned.
     * `OCCUPANCY_RATE` values are between 0.3 and 1. 

##### Tickets Data - Profiling

In [5]:
# Profiling Tickets Data
if tickets_raw_df is not None:
    print("\n--- Tickets Raw Data ---")
    print("Head:")
    print(tickets_raw_df.head())
    print("\nInfo:")
    tickets_raw_df.info()
    print("\nDescribe:")
    print(tickets_raw_df.describe(include='all'))
    # Generate ydata-profiling report
    # try:
    #     print(f"Generating profiling report for Tickets data, saving to {config.PROFILING_REPORT_TICKETS}")
    #     profile_tickets = ProfileReport(tickets_raw_df, title="Tickets Data Profiling Report")
    #     profile_tickets.to_file(config.PROFILING_REPORT_TICKETS)
    #     print(f"Tickets profiling report saved to {config.PROFILING_REPORT_TICKETS}")
    # except Exception as e:
    #     print(f"Could not generate ydata-profiling report for tickets: {e}")
else:
    print("Tickets data (tickets_raw_df) is None, was not loaded correctly. Skipping profiling.")


--- Tickets Raw Data ---
Head:
        ITIN_ID  YEAR  QUARTER ORIGIN ORIGIN_COUNTRY ORIGIN_STATE_ABR  \
0  201912723049  2019        1    ABI             US               TX   
1  201912723085  2019        1    ABI             US               TX   
2  201912723491  2019        1    ABI             US               TX   
3  201912723428  2019        1    ABI             US               TX   
4  201912723509  2019        1    ABI             US               TX   

  ORIGIN_STATE_NM  ROUNDTRIP REPORTING_CARRIER  PASSENGERS ITIN_FARE  \
0           Texas       1.00                MQ        1.00     736.0   
1           Texas       1.00                MQ        1.00     570.0   
2           Texas       1.00                MQ        1.00     564.0   
3           Texas       1.00                MQ        1.00     345.0   
4           Texas       0.00                MQ        1.00     309.0   

  DESTINATION  
0         DAB  
1         COS  
2         MCO  
3         LGA  
4         MGM  


Initial Hypotheses and Observations:
 * Tickets Data:
     * Analysis requires filtering for `ROUNDTRIP == 1` and data from `1Q2019` (`YEAR == 2019`, `QUARTER == 1`).
     * `ITIN_FARE` is a crucial column for profitability analysis; its distribution and potential outliers will be important.
     * This dataset is explicitly mentioned as "sample data," so it might not provide fare information for all routes present in the Flights dataset. This could lead to challenges in the join process.

##### Airport Codes Data - Profiling

In [6]:
# Profiling Airport Codes Data
if airport_codes_raw_df is not None:
    print("\n--- Airport Codes Raw Data ---")
    print("Head:")
    print(airport_codes_raw_df.head())
    print("\nInfo:")
    airport_codes_raw_df.info()
    print("\nDescribe:")
    print(airport_codes_raw_df.describe(include='all'))
    # Generate ydata-profiling report
    # try:
    #     print(f"Generating profiling report for Airport Codes data, saving to {config.PROFILING_REPORT_AIRPORTS}")
    #     profile_airports = ProfileReport(airport_codes_raw_df, title="Airport Codes Data Profiling Report")
    #     profile_airports.to_file(config.PROFILING_REPORT_AIRPORTS)
    #     print(f"Airport codes profiling report saved to {config.PROFILING_REPORT_AIRPORTS}")
    # except Exception as e:
    #     print(f"Could not generate ydata-profiling report for airport codes: {e}")
else:
    print("Airport codes data (airport_codes_raw_df) is None, was not loaded correctly. Skipping profiling.")


--- Airport Codes Raw Data ---
Head:
            TYPE                                NAME  ELEVATION_FT CONTINENT  \
0       heliport                   Total Rf Heliport         11.00       NaN   
1  small_airport                Aero B Ranch Airport       3435.00       NaN   
2  small_airport                        Lowell Field        450.00       NaN   
3  small_airport                        Epps Airpark        820.00       NaN   
4         closed  Newport Hospital & Clinic Heliport        237.00       NaN   

  ISO_COUNTRY  MUNICIPALITY IATA_CODE                            COORDINATES  
0          US      Bensalem       NaN     -74.93360137939453, 40.07080078125  
1          US         Leoti       NaN                 -101.473911, 38.704022  
2          US  Anchor Point       NaN            -151.695999146, 59.94919968  
3          US       Harvest       NaN  -86.77030181884766, 34.86479949951172  
4          US       Newport       NaN                    -91.254898, 35.6087  

Info:


Initial Hypotheses and Observations:
 * Airport Codes Data:
     * Filtering will be necessary for `ISO_COUNTRY == 'US'` and `TYPE` in `['medium_airport', 'large_airport']`.
     * `IATA_CODE` will serve as the primary key for joining with Flights and Tickets data. Uniqueness of `IATA_CODE` after filtering will be important.

 *General Observations:*
 * Data quality issues such as missing values, outliers, incorrect data types, and inconsistencies are expected and will be addressed systematically.
 * The join strategy between these datasets will be critical. Inner joins might lead to data loss if keys don't match perfectly (e.g., an airport in Flights not present in filtered Airport Codes, or a route in Flights not having sample fare data in Tickets).
 * The `dq_utils` module will be used to log metrics at each step, providing a traceable record of data transformations and quality checks.


 ### Task 1.2: Data Cleaning & Quality Assurance
 - For each significant cleaning step, log DQ metrics before and after the operation using `dq_utils`.
 - Use `clean_utils.py` for cleaning operations (handling missing values, data type conversions, filtering based on requirements).
 - Document all cleaning steps, rationale, and at least 3 data quality insights.
 - All the rejected records have to be captured in a seperate CSV files for further manual analysis.

 #### Cleaning Flights Data

Flights Data Cleaning:

* Steps:
    * Select Relevant Columns: Keeps only essential flight information columns (e.g., date, origin, destination, delays, distance, occupancy, cancellation status).
    * Convert Data Types: Ensures dates are date objects, numerical fields (like distance, airtime, delays, occupancy) are numbers, and identifiers (like origin, destination) are strings. Unparseable values become 'missing'.
    * Filter Cancelled Flights: Removes rows where the flight is marked as cancelled or its cancellation status is unclear/missing.
    * Handle Missing/Unparseable Values:
        * Imputes missing departure and arrival delays with 0.
        * Filters out rows if other critical required columns (like flight date, airtime after type conversion) still have missing/unparseable values.
    * Validate Occupancy Rate: Filters out rows where the occupancy rate is not between 0 and 1 (inclusive) or is missing.
    * Handle Outliers: Applies the configured outlier handling (impute with mean, filter, or none) to DEP_DELAY, ARR_DELAY, AIR_TIME, and DISTANCE.
    * Log & Save Rejects: All filtered rows are saved to rejected_flights_data.csv with reasons.

In [7]:
if flights_raw_df is not None:
    print("\n--- Cleaning Flights Data ---")
    # The clean_flights_data function handles missing values (dropping critical NAs, filling delay NAs),
    # converts data types, filters out cancelled flights, and validates occupancy rate. DQ metrics are logged.
    flights_cleaned_df = clean_utils.clean_flights_data(flights_raw_df.copy())
    if flights_cleaned_df is not None and not flights_cleaned_df.empty:
        print("\nCleaned Flights Info:")
        flights_cleaned_df.info()
        print(f"\nShape of cleaned flights: {flights_cleaned_df.shape}")
        # Verify filters
        print(f"Cancelled values in cleaned flights data: {flights_cleaned_df[config.COL_FL_CANCELLED].unique()}") # Should be [0]
        print(f"Min/Max Occupancy Rate: {flights_cleaned_df[config.COL_FL_OCCUPANCY].min()} / {flights_cleaned_df[config.COL_FL_OCCUPANCY].max()}")
    elif flights_cleaned_df is not None and flights_cleaned_df.empty:
        print("Flights data cleaning resulted in an empty DataFrame. Check filters (e.g., cancellation filter) and raw data.")
    else: # flights_cleaned_df is None
        print("Flights data cleaning failed or returned None.")
else:
    print("Flights raw data (flights_raw_df) is not available for cleaning.")
    flights_cleaned_df = pd.DataFrame()

step_name = "flights_after_clean"
file_name = f"{step_name}.csv"
output_file_path = os.path.join(config.STEP_OUTPUT_DIR, file_name)
batch_timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M")

flights_cleaned_df.loc[:, 'batch_id'] = batch_timestamp

flights_cleaned_df.to_csv(output_file_path, index=False)
print(f"Saved DataFrame to: {output_file_path}")


--- Cleaning Flights Data ---
Step: Filtered Flights to required columns. Kept 10 columns.
Excluded columns at cleaning start for Flights: ['OP_CARRIER', 'TAIL_NUM', 'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_NAME', 'DEST_AIRPORT_ID', 'DEST_CITY_NAME']
Filtered out 51614 cancelled/unclear flights.

--- Handling Missing/Unusual Values for Flights ---
Filtered 4900 rows (NaNs in 'FL_DATE').
Imputed 4367 NaNs in 'ARR_DELAY'.
Filtered 4367 rows (NaNs in 'AIR_TIME').
Total rows filtered (missing/unparseable) from Flights: 9267
Detected 244571 outliers in 'DEP_DELAY'.
Imputed 244571 outliers in 'DEP_DELAY' with mean: -1.52.
Detected 174562 outliers in 'ARR_DELAY'.
Imputed 174562 outliers in 'ARR_DELAY' with mean: -5.60.
Detected 93850 outliers in 'AIR_TIME'.
Imputed 93850 outliers in 'AIR_TIME' with mean: 98.49.
Detected 98297 outliers in 'DISTANCE'.
Imputed 98297 outliers in 'DISTANCE' with mean: 678.19.
Saved 56336 unique rejected rows for Flights_Rejects to /Users/Rajz/Documents/Job Search/Capita

#### Cleaning Tickets Data

* Steps:
    * Select Relevant Columns: Keeps only essential ticket information (e.g., year, quarter, origin, destination, roundtrip status, itinerary fare).
    * Convert Data Types: Ensures numerical fields (like fare, year, quarter) are numbers and identifiers are strings. Unparseable values become 'missing'.
    * Filter by Business Rules:
        * Removes tickets that are not round trips (ROUNDTRIP != 1) or where roundtrip status is unclear/missing.
        * Removes tickets not within the specified analysis period (1Q2019).
    * Handle Missing/Unparseable Values: Filters out rows if any remaining required columns (like itinerary fare, origin, destination after type conversion) have missing/unparseable values.
    * Validate Itinerary Fare: Filters out tickets with a non-positive (<=0) itinerary fare.
    * Handle Outliers: Applies the configured outlier handling to ITIN_FARE.
    * Log & Save Rejects: All filtered rows are saved to rejected_tickets_data.csv with reasons.

In [8]:
if tickets_raw_df is not None:
    print("\n--- Cleaning Tickets Data ---")
    # The clean_tickets_data function handles missing values, filters for 1Q2019 roundtrip tickets,
    # converts data types, and filters non-positive fares. DQ metrics are logged within.
    tickets_cleaned_df = clean_utils.clean_tickets_data(tickets_raw_df.copy())
    if tickets_cleaned_df is not None and not tickets_cleaned_df.empty:
        print("\nCleaned Tickets Info:")
        tickets_cleaned_df.info()
        print(f"\nShape of cleaned tickets: {tickets_cleaned_df.shape}")
        # Verify filters
        print(f"Years in cleaned tickets data: {tickets_cleaned_df[config.COL_TK_YEAR].unique()}")
        print(f"Quarters in cleaned tickets data: {tickets_cleaned_df[config.COL_TK_QUARTER].unique()}")
        print(f"Roundtrip values in cleaned tickets data: {tickets_cleaned_df[config.COL_TK_ROUNDTRIP].unique()}")
        print(f"Min ITIN_FARE in cleaned tickets: {tickets_cleaned_df[config.COL_TK_ITIN_FARE].min()}")
    elif tickets_cleaned_df is not None and tickets_cleaned_df.empty:
        print("Tickets data cleaning resulted in an empty DataFrame. This is possible if no 1Q2019 roundtrip tickets exist or all had issues.")
        print("Profitability analysis will be significantly impacted if ticket data is empty.")
    else: # tickets_cleaned_df is None
        print("Tickets data cleaning failed or returned None.")
else:
    print("Tickets raw data (tickets_raw_df) is not available for cleaning.")
    tickets_cleaned_df = pd.DataFrame()

step_name = "tickets_after_clean"
file_name = f"{step_name}.csv"
output_file_path = os.path.join(config.STEP_OUTPUT_DIR, file_name)
batch_timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M")

tickets_cleaned_df.loc[:, 'batch_id'] = batch_timestamp

tickets_cleaned_df.to_csv(output_file_path, index=False)
print(f"Saved DataFrame to: {output_file_path}")


--- Cleaning Tickets Data ---
Step: Filtered Tickets to required columns. Kept 7 columns.
Excluded columns at cleaning start for Tickets: ['ITIN_ID', 'ORIGIN_COUNTRY', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'REPORTING_CARRIER']
Filtered 458685 non-roundtrip tickets.
Filtered 1197 rows (NaNs in 'PASSENGERS' for Tickets).
Filtered 1842 rows (NaNs in 'ITIN_FARE' for Tickets).
Total rows filtered (missing/unparseable) from Tickets: 3039
Filtered 6370 tickets (non-positive fare).
Detected 30846 outliers in 'ITIN_FARE'.
Imputed 30846 outliers in 'ITIN_FARE' with mean: 430.08.
Saved 414758 unique rejected rows for Tickets_Rejects to /Users/Rajz/Documents/Job Search/Capital One/airline_data_challenge_submission/logs/rejected_tickets_data.csv
Finished cleaning Tickets data. Final shape: (699191, 7)

Cleaned Tickets Info:
<class 'pandas.core.frame.DataFrame'>
Index: 699191 entries, 0 to 1167284
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------

#### Cleaning Airport Codes Data

* Steps:
    * Select Relevant Columns: Keeps essential airport information (IATA code, type, name, country, coordinates).
    * Convert Data Types: Ensures key identifiers and filterable fields (like IATA code, type, country, coordinates) are treated as strings initially for reliable processing.
    * Parse Coordinates: Splits the COORDINATES string into separate LATITUDE and LONGITUDE numeric columns. Rows with malformed or unparseable coordinates are filtered out.
    * Handle Missing Critical Info: Filters out airports missing essential identifiers needed for subsequent filtering (IATA_CODE, TYPE, ISO_COUNTRY).
    * Filter by Business Rules:
        * Removes airports not in the 'US' (ISO_COUNTRY != 'US').
        * Removes airports not classified as 'medium_airport' or 'large_airport'.
        * Removes duplicate airport entries based on IATA_CODE, keeping only the first occurrence.
    * Handle Remaining Missing Values: Filters out rows if any other required columns (like NAME, or the newly created LATITUDE/LONGITUDE) have missing values.
    * Outlier Handling: Typically not applied to descriptive airport fields in this context.
    * Log & Save Rejects: All filtered rows are saved to rejected_airports_data.csv with reasons.

In [9]:
if airport_codes_raw_df is not None:
    print("\n--- Cleaning Airport Codes Data ---")
    # The clean_airport_codes_data function handles missing values, filters for US medium/large airports,
    # and checks for duplicate IATA codes. DQ metrics are logged within the function.
    airport_codes_cleaned_df = clean_utils.clean_airport_codes_data(airport_codes_raw_df.copy()) # Use .copy() to avoid SettingWithCopyWarning on the raw df
    if airport_codes_cleaned_df is not None and not airport_codes_cleaned_df.empty:
        print("\nCleaned Airport Codes Info:")
        airport_codes_cleaned_df.info()
        print(f"\nShape of cleaned airport codes: {airport_codes_cleaned_df.shape}")
        print(f"Unique IATA codes in cleaned data: {airport_codes_cleaned_df[config.COL_AIRPORT_IATA].nunique()}")
        print(f"Types of airports in cleaned data: {airport_codes_cleaned_df[config.COL_AIRPORT_TYPE].unique()}")
        print(f"Countries in cleaned data: {airport_codes_cleaned_df[config.COL_AIRPORT_ISO_COUNTRY].unique()}")
    elif airport_codes_cleaned_df is not None and airport_codes_cleaned_df.empty:
        print("Airport codes cleaning resulted in an empty DataFrame. Check filters and raw data.")
    else: # airport_codes_cleaned_df is None
        print("Airport codes cleaning failed or returned None.")
else:
    print("Airport codes raw data (airport_codes_raw_df) is not available for cleaning.")
    airport_codes_cleaned_df = pd.DataFrame() # Ensure it's an empty DF to prevent errors later

step_name = "airports_after_clean"
file_name = f"{step_name}.csv"
output_file_path = os.path.join(config.STEP_OUTPUT_DIR, file_name)
batch_timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M")

airport_codes_cleaned_df.loc[:, 'batch_id'] = batch_timestamp

airport_codes_cleaned_df.to_csv(output_file_path, index=False)
print(f"Saved DataFrame to: {output_file_path}")


--- Cleaning Airport Codes Data ---
Step: Filtered Airport_Codes to required columns. Kept 5 columns.
Excluded columns at cleaning start for Airport_Codes: ['ELEVATION_FT', 'CONTINENT', 'MUNICIPALITY']

--- Parsing Coordinates for Airport_Codes ---
Successfully parsed coordinates into LATITUDE and LONGITUDE.
Filtered 32559 non-US airports.
Filtered 21952 invalid airport types.
Filtered 36 duplicate IATA airports.

--- Handling Missing/Unusual Values for Airport_Codes (Remaining Columns) ---

--- Outlier handling not applied to Airport_Codes descriptive fields ---
Saved 54438 unique rejected rows for Airport_Codes_Rejects to /Users/Rajz/Documents/Job Search/Capital One/airline_data_challenge_submission/logs/rejected_airports_data.csv
Finished cleaning Airport_Codes data. Final shape: (822, 7)

Cleaned Airport Codes Info:
<class 'pandas.core.frame.DataFrame'>
Index: 822 entries, 6194 to 39286
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       ---

### Task 1.3: Data Transformation & Feature Engineering
 - For key transformation steps (e.g., joins), log DQ metrics before and after operations using `dq_utils`.
 - Use `feature_engineering_utils.py` for transformations like creating route columns, joining datasets, and calculating detailed cost/revenue figures per flight.
 - Document metadata for newly engineered fields (this is primarily done via docstrings in `.py` files and `config.py`, and summarized here).

* Steps:
    * Initial Logging: Records the start of the feature engineering process.
    * Create Route Identifiers: Generates ROUTE (Origin-Destination) and CANONICAL_ROUTE_PAIR (standardized round-trip) columns.
    * Join Flights with Airport Details: Merges flight data with airport data (name, type, lat/lon) for both origin and destination.
    * Join with Aggregated Ticket Data: Attaches average route fares (from sample ticket data) to flight records.
    * Calculate Flight Costs: Computes various operational costs (fuel, airport, delay, etc.) for each flight leg.
    * Calculate Flight Revenue: Estimates ticket and baggage revenue for each flight leg based on occupancy and fares.
    * Calculate Profit (per leg): Determines operational profit for each flight leg by subtracting total costs from total revenue.
    * Final ABT Creation & Logging: Consolidates all transformations into the Analytical Base Table and logs its final status.

In [10]:
# Ensure all cleaned dataframes are not None and preferably not empty before proceeding
abt_df = pd.DataFrame() # Initialize to empty DataFrame

# Check if prerequisite dataframes are valid
prereq_valid = True
if flights_cleaned_df is None or flights_cleaned_df.empty:
    print("Cleaned flights data is missing or empty. Cannot proceed with feature engineering.")
    prereq_valid = False
if tickets_cleaned_df is None: # tickets_cleaned_df can be empty if no 1Q2019 data, but not None
    print("Cleaned tickets data is None (failed cleaning). Cannot proceed with feature engineering.")
    prereq_valid = False
if airport_codes_cleaned_df is None or airport_codes_cleaned_df.empty:
    print("Cleaned airport codes data is missing or empty. Cannot proceed with feature engineering.")
    prereq_valid = False

if prereq_valid:
    print("\n--- Starting Feature Engineering ---")
    # The create_analytical_base_table function orchestrates joins and feature calculations.
    # It logs DQ metrics internally for joins and new feature creation.
    abt_df = feature_engineering_utils.create_analytical_base_table(
        flights_cleaned_df,
        tickets_cleaned_df, # This can be an empty DataFrame if no valid tickets were found
        airport_codes_cleaned_df
    )

    if abt_df is not None and not abt_df.empty:
        print("\n--- Analytical Base Table (ABT) Created ---")
        print("ABT Head:")
        print(abt_df.head())
        print("\nABT Info:")
        abt_df.info()
        print(f"\nShape of ABT: {abt_df.shape}")

        # Display some stats for key engineered features
        print("\nStats for key engineered features in ABT:")
        key_features_to_check = [
            'ROUTE', 'CANONICAL_ROUTE_PAIR', 'ORIGIN_AIRPORT_TYPE', 'DEST_AIRPORT_TYPE',
            'AVG_ROUTE_ITIN_FARE', 'COST_FUEL_ETC', 'COST_DEPRECIATION_ETC',
            'COST_AIRPORT_OPERATIONAL', 'COST_DEP_DELAY', 'COST_ARR_DELAY',
            'TOTAL_FLIGHT_COST', 'CALCULATED_PASSENGERS',
            'TICKET_REVENUE_PER_LEG', 'BAGGAGE_REVENUE_PER_LEG',
            'TOTAL_FLIGHT_REVENUE', 'PROFIT_PER_LEG'
        ]
        for col in key_features_to_check:
            if col in abt_df.columns:
                print(f"\n--- Stats for {col} ---")
                print(abt_df[col].describe(include='all'))
                print(f"Missing values in {col}: {abt_df[col].isnull().sum()}")
            else:
                print(f"Column {col} was expected but not found in ABT.")
    elif abt_df is not None and abt_df.empty:
         print("Analytical Base Table (ABT) was created but is empty. This likely means joins resulted in no matching records.")
         print("Review join conditions, filters in cleaning, and DQ logs from feature_engineering_utils.")
    else: # abt_df is None
        print("Analytical Base Table (ABT) creation failed or returned None.")
else:
    print("One or more cleaned dataframes are not suitable for feature engineering. Skipping ABT creation.")
    # Ensure abt_df is an empty DataFrame if it couldn't be created
    if 'abt_df' not in locals() or abt_df is None:
         abt_df = pd.DataFrame()

step_name = "featured_data"
file_name = f"{step_name}.csv"
output_file_path = os.path.join(config.STEP_OUTPUT_DIR, file_name)
batch_timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M")

abt_df.loc[:, 'batch_id'] = batch_timestamp

abt_df.to_csv(output_file_path, index=False)
print(f"Saved DataFrame to: {output_file_path}")


--- Starting Feature Engineering ---
--- Starting Feature Engineering Phase ---
Flights after inner join with origin airports: 1838774 rows.
Flights after inner join with destination airports: 1823296 rows.
Joined with aggregated ticket data. 1804570 flights got fare, 18726 did not.
--- Feature Engineering Phase Complete. ABT created with shape: (1823296, 35) ---

--- Analytical Base Table (ABT) Created ---
ABT Head:
     FL_DATE OP_CARRIER_FL_NUM ORIGIN DESTINATION  DEP_DELAY  ARR_DELAY  \
0 2019-03-02              4591    RSW         CLE      -8.00      -6.00   
1 2019-03-02              3231    RSW         CMH       1.00       5.00   
2 2019-03-02              3383    RSW         CMH       0.00       4.00   
3 2019-03-02              5498    RSW         CMH      11.00      14.00   
4 2019-03-02              6933    RSW         DAL       0.00     -17.00   

   CANCELLED  AIR_TIME  DISTANCE  OCCUPANCY_RATE          batch_id    ROUTE  \
0          0    143.00   1025.00            0.97

 **Metadata for Newly Engineered Fields (examples from `config.py` and `feature_engineering_utils.py`):**

 ### Metadata for Newly Engineered Columns in ABT

| New Column Name          | Data Type (in ABT) | Description                                                                                                | Source/Phase Added                                  |
| :----------------------- | :----------------- | :--------------------------------------------------------------------------------------------------------- | :-------------------------------------------------- |
| `ROUTE`                  | object (string)    | Concatenation of Origin and Destination airport codes (e.g., "JFK-ORD").                                   | `feature_engineering_utils.py` (create_route_columns) |
| `CANONICAL_ROUTE_PAIR`   | object (string)    | Standardized round-trip identifier (e.g., "JFK-ORD" for both JFK-ORD and ORD-JFK).                         | `feature_engineering_utils.py` (create_route_columns) |
| `ORIGIN_AIRPORT_TYPE`    | object (string)    | Type of the origin airport (e.g., 'large_airport', 'medium_airport').                                      | `feature_engineering_utils.py` (join_flights_with_airports) |
| `ORIGIN_AIRPORT_NAME`    | object (string)    | Full name of the origin airport.                                                                           | `feature_engineering_utils.py` (join_flights_with_airports) |
| `ORIGIN_LATITUDE`        | Float64            | Latitude of the origin airport.                                                                            | `feature_engineering_utils.py` (join_flights_with_airports) / `clean_utils.py` (airport cleaning) |
| `ORIGIN_LONGITUDE`       | Float64            | Longitude of the origin airport.                                                                           | `feature_engineering_utils.py` (join_flights_with_airports) / `clean_utils.py` (airport cleaning) |
| `DEST_AIRPORT_TYPE`      | object (string)    | Type of the destination airport.                                                                           | `feature_engineering_utils.py` (join_flights_with_airports) |
| `DEST_AIRPORT_NAME`      | object (string)    | Full name of the destination airport.                                                                      | `feature_engineering_utils.py` (join_flights_with_airports) |
| `DEST_LATITUDE`          | Float64            | Latitude of the destination airport.                                                                       | `feature_engineering_utils.py` (join_flights_with_airports) / `clean_utils.py` (airport cleaning) |
| `DEST_LONGITUDE`         | Float64            | Longitude of the destination airport.                                                                      | `feature_engineering_utils.py` (join_flights_with_airports) / `clean_utils.py` (airport cleaning) |
| `AVG_ROUTE_ITIN_FARE`    | Float64            | Average round-trip itinerary fare per person for the route, derived from sample Tickets data.                | `feature_engineering_utils.py` (join_with_tickets_data) |
| `COST_FUEL_ETC`          | Float64 or Int64   | Cost for fuel, oil, maintenance, and crew per flight leg.                                                  | `feature_engineering_utils.py` (calculate_flight_costs) |
| `COST_DEPRECIATION_ETC`  | Float64            | Cost for depreciation, insurance, and other items per flight leg.                                          | `feature_engineering_utils.py` (calculate_flight_costs) |
| `COST_AIRPORT_OPERATIONAL`| int64 or float64   | Landing fee at the destination airport for a flight leg.                                                   | `feature_engineering_utils.py` (calculate_flight_costs) |
| `DEP_DELAY_CHARGEABLE`   | Float64            | Departure delay minutes exceeding the free allowance, used for cost calculation.                           | `feature_engineering_utils.py` (calculate_flight_costs) |
| `COST_DEP_DELAY`         | Float64            | Cost incurred due to departure delays.                                                                     | `feature_engineering_utils.py` (calculate_flight_costs) |
| `ARR_DELAY_CHARGEABLE`   | Float64            | Arrival delay minutes exceeding the free allowance, used for cost calculation.                             | `feature_engineering_utils.py` (calculate_flight_costs) |
| `COST_ARR_DELAY`         | Float64            | Cost incurred due to arrival delays.                                                                       | `feature_engineering_utils.py` (calculate_flight_costs) |
| `TOTAL_FLIGHT_COST`      | Float64            | Sum of all operational cost components for a single flight leg.                                            | `feature_engineering_utils.py` (calculate_flight_costs) |
| `CALCULATED_PASSENGERS`  | int64              | Estimated number of passengers on a flight based on occupancy rate.                                        | `feature_engineering_utils.py` (calculate_flight_revenue) |
| `TICKET_REVENUE_PER_LEG` | Float64            | Estimated ticket revenue for a single flight leg (AVG_ROUTE_ITIN_FARE/2 * passengers).                       | `feature_engineering_utils.py` (calculate_flight_revenue) |
| `BAGGAGE_REVENUE_PER_LEG`| float64            | Estimated baggage revenue for a single flight leg.                                                         | `feature_engineering_utils.py` (calculate_flight_revenue) |
| `TOTAL_FLIGHT_REVENUE`   | Float64            | Sum of ticket and baggage revenue for a single flight leg.                                                 | `feature_engineering_utils.py` (calculate_flight_revenue) |
| `PROFIT_PER_LEG`         | Float64            | Operational profit for a single flight leg (TOTAL_FLIGHT_REVENUE - TOTAL_FLIGHT_COST).                     | `feature_engineering_utils.py` (calculate_flight_revenue) |
| `AVG_TOTAL_DELAY`        | Float64            | Sum of average departure and arrival delay for a route (created in analysis_utils for scoring).            | `analysis_utils.py` (recommend_routes_advanced_scoring) |
| `PROFIT_PER_FLIGHT`      | Float64            | Total profit for a canonical route pair divided by the number of round trip flights on that pair.          | `analysis_utils.py` (identify_most_profitable_round_trip_routes) |
| `BREAKEVEN_ROUND_TRIP_FLIGHTS` | Float64 or Int64 | Number of round trip flights needed to cover the $90M airplane cost for a route.                       | `analysis_utils.py` (calculate_breakeven_flights) |
| `COMPOSITE_SCORE`        | float64            | Overall score assigned to a route by the advanced recommendation model.                                    | `analysis_utils.py` (recommend_routes_advanced_scoring) |
| `RANK`                   | int64              | Rank of the route based on the composite score.                                                            | `analysis_utils.py` (recommend_routes_advanced_scoring) |
| `IS_RECOMMENDED`         | boolean            | Flag indicating if a route is part of the final recommendations (used for Tableau export).                 | Notebook (Tableau Export Cell)                      |
| `BATCH_ID`               | object (string)    | Timestamp indicating when the data (e.g., intermediate CSV) was generated/processed for that specific run. | Notebook (CSV Saving Cells) / `clean_utils.py` (for rejected data) |
| `FILTER_REASON`          | object (string)    | Column added to rejected data CSVs explaining why a row was filtered out.                                  | `clean_utils.py` (_save_rejected_data)              |
| `REJECT_RUN_ID`          | object (string)    | Run ID associated with the cleaning run that produced the rejected row.                                    | `clean_utils.py` (_save_rejected_data)              |
| `REJECT_TIMESTAMP`       | object (string)    | Timestamp of when the row was rejected during the cleaning process.                                        | `clean_utils.py` (_save_rejected_data)              |


In [11]:
# Save the complete DQ log at the end of Phase 1 (Data Foundation)
# This includes logs from loading, cleaning, and feature engineering.
dq_utils.save_dq_log_to_csv(config.DQ_LOG_FILE_PATH)
print(f"Full DQ log for Phase 1 saved to {config.DQ_LOG_FILE_PATH}")

# Display a summary of the DQ log from the current run
dq_utils.display_dq_summary()

DQ log for Run ID 49ec5f94-94e7-4896-94ba-67fc53ec3155 saved to /Users/Rajz/Documents/Job Search/Capital One/airline_data_challenge_submission/logs/dq_metrics.csv
Full DQ log for Phase 1 saved to /Users/Rajz/Documents/Job Search/Capital One/airline_data_challenge_submission/logs/dq_metrics.csv

--- Data Quality Log Summary ---
Run ID: 49ec5f94-94e7-4896-94ba-67fc53ec3155
Total DQ metrics logged: 66
Last 5 entries:
              Timestamp                                 RunID  \
61  2025-05-13 21:40:52  49ec5f94-94e7-4896-94ba-67fc53ec3155   
62  2025-05-13 21:40:52  49ec5f94-94e7-4896-94ba-67fc53ec3155   
63  2025-05-13 21:40:52  49ec5f94-94e7-4896-94ba-67fc53ec3155   
64  2025-05-13 21:40:52  49ec5f94-94e7-4896-94ba-67fc53ec3155   
65  2025-05-13 21:40:52  49ec5f94-94e7-4896-94ba-67fc53ec3155   

                                        Phase  \
61  Data Transformation & Feature Engineering   
62  Data Transformation & Feature Engineering   
63  Data Transformation & Feature Engineerin

## Phase 2: Analytical Insights & Visualization
 

1. Identify Busiest Routes: Determine and list the top 10 busiest round-trip routes based on flight volume.
2. Identify Most Profitable Routes: Determine and list the top 10 most profitable round-trip routes (pre-airplane cost), along with their key financial and operational metrics.
3. Calculate Breakeven for Profitable Routes: For all routes identified as profitable, calculate their breakeven flight volume considering the airplane investment cost.
4. Recommend Top Routes (Advanced Scoring): Apply a multi-factored scoring model (using profitability, demand, operational efficiency, and investment viability metrics) to select and rank the top 5 routes for investment.
5. Display Breakeven for Recommended Routes: Specifically show the breakeven analysis details for the 5 finally recommended routes.
6. Prepare Data for Visualization: Export the key analytical datasets (e.g., route metrics, recommendations, DQ logs) for use in Tableau or other BI tools.



 ### Task 2.1: Answer Core Business Questions (Calculations in Python)
 1.  Determine the 10 busiest round trip routes by number of round trip flights.
 2.  Determine the 10 most profitable round trip routes (pre-airplane cost) and their associated metrics.

 #### Q1: The 10 Busiest Round Trip Routes

In [12]:
busiest_routes_df = pd.DataFrame() 
if 'abt_df' in locals() and abt_df is not None and not abt_df.empty:
    print("\n--- Q1: Identifying 10 Busiest Round Trip Routes ---")
    if 'CANONICAL_ROUTE_PAIR' not in abt_df.columns:
        print("Error: 'CANONICAL_ROUTE_PAIR' column not found in ABT. Cannot determine busiest routes.")
    else:
        busiest_routes_df = analysis_utils.identify_busiest_round_trip_routes(abt_df.copy(), top_n=10)
        if not busiest_routes_df.empty:
            # plt.figure(figsize=(12, 7))
            # sns.barplot(x='ROUND_TRIP_FLIGHTS', y='CANONICAL_ROUTE_PAIR', data=busiest_routes_df, palette='viridis', hue='CANONICAL_ROUTE_PAIR', dodge=False, legend=False)
            # plt.title('Top 10 Busiest Round Trip Routes (1Q2019)')
            # plt.xlabel('Number of Round Trip Flights'); plt.ylabel('Round Trip Route')
            # plt.tight_layout(); plt.show()
            # Save intermediate output
            step_name = "04_busiest_routes"
            file_name = f"{step_name}.csv"; output_file_path = os.path.join(config.STEP_OUTPUT_DIR, file_name)
            df_to_save = busiest_routes_df.copy()
            df_to_save.loc[:, config.BATCH_ID_COLUMN] = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
            df_to_save.to_csv(output_file_path, index=False); print(f"Saved DataFrame to: {output_file_path}")
        else:
            print("Could not determine busiest routes.")
else:
    print("ABT is empty or not available. Skipping Q1 analysis (Busiest Routes).")


--- Q1: Identifying 10 Busiest Round Trip Routes ---

--- Top 10 Busiest Round Trip Routes ---
     CANONICAL_ROUTE_PAIR  ROUND_TRIP_FLIGHTS
2156              LAX-SFO                4166
2197              LGA-ORD                3573
2060              LAS-LAX                3253
2004              JFK-LAX                3149
2155              LAX-SEA                2496
546               BOS-LGA                2408
1766              HNL-OGG                2396
2611              PDX-SEA                2385
190               ATL-MCO                2353
186               ATL-LGA                2294
Saved DataFrame to: /Users/Rajz/Documents/Job Search/Capital One/airline_data_challenge_submission/output_steps/04_busiest_routes.csv


#### Q2: The 10 Most Profitable Round Trip Routes

In [13]:
most_profitable_routes_df = pd.DataFrame()
if 'abt_df' in locals() and abt_df is not None and not abt_df.empty:
    print("\n--- Q2: Identifying 10 Most Profitable Round Trip Routes ---")
    # ... (checks for required columns and AVG_ROUTE_ITIN_FARE as in your notebook) ...
    most_profitable_routes_df = analysis_utils.identify_most_profitable_round_trip_routes(abt_df.copy(), top_n=10) # top_n can be larger if we want to score more routes
    if not most_profitable_routes_df.empty:
        # Save intermediate output
        step_name = "05_most_profitable_routes"
        file_name = f"{step_name}.csv"; output_file_path = os.path.join(config.STEP_OUTPUT_DIR, file_name)
        df_to_save = most_profitable_routes_df.copy()
        df_to_save.loc[:, config.BATCH_ID_COLUMN] = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
        df_to_save.to_csv(output_file_path, index=False); print(f"Saved DataFrame to: {output_file_path}")
    else:
        print("Could not determine most profitable routes.")
else:
    print("ABT is empty or not available. Skipping Q2 analysis (Most Profitable Routes).")



--- Q2: Identifying 10 Most Profitable Round Trip Routes ---

--- Top 10 Most Profitable Round Trip Routes (excluding upfront airplane cost) ---
     CANONICAL_ROUTE_PAIR  TOTAL_PROFIT  PROFIT_PER_FLIGHT  TOTAL_REVENUE  \
1985              JFK-LAX   99252298.23           31518.67   202069077.44   
1111              DCA-ORD   73399227.77           39804.35   131355101.85   
1095              DCA-LGA   69124622.46           41194.65   109716451.58   
540               BOS-LGA   66398343.46           27574.06   123197970.38   
129               ATL-CLT   65900851.44           42876.29   103257592.76   
2135              LAX-SFO   63654207.51           15279.45   173868501.29   
833               CLT-GSP   61872487.00           80042.03    78516796.50   
2011              JFK-SFO   57715834.19           31197.75   118210322.42   
1286              DFW-IAH   57277348.38           38779.52    93256592.66   
2176              LGA-ORD   56426861.22           15792.57   177049691.46   

      

### Task: Calculate Breakeven for Profitable Routes (Input for Q3 Recommendation)
 This step calculates the breakeven point for all routes identified as potentially profitable. This information will then be used in the advanced scoring model for recommendations. This addresses part of Q4 from the problem statement upfront.

In [14]:
breakeven_for_all_profitable_df = pd.DataFrame()
if 'most_profitable_routes_df' in locals() and not most_profitable_routes_df.empty:
    print("\n--- Calculating Breakeven Analysis for All Profitable Routes ---")
    # We use most_profitable_routes_df for both arguments as it contains the list of routes
    # and the necessary 'PROFIT_PER_FLIGHT' detail.
    breakeven_for_all_profitable_df = analysis_utils.calculate_breakeven_flights(
        most_profitable_routes_df, # Provides CANONICAL_ROUTE_PAIR for routes to analyze
        most_profitable_routes_df  # Provides the detailed profit metrics including PROFIT_PER_FLIGHT
    )
    if not breakeven_for_all_profitable_df.empty:
        print(f"\nBreakeven Analysis Calculated for {len(breakeven_for_all_profitable_df)} profitable routes.")
        print(breakeven_for_all_profitable_df.head())
        # Save intermediate output
        step_name = "06_breakeven_for_all_profitable_routes"
        file_name = f"{step_name}.csv"; output_file_path = os.path.join(config.STEP_OUTPUT_DIR, file_name)
        df_to_save = breakeven_for_all_profitable_df.copy()
        df_to_save.loc[:, config.BATCH_ID_COLUMN] = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
        df_to_save.to_csv(output_file_path, index=False); print(f"Saved DataFrame to: {output_file_path}")
    else:
        print("Could not perform breakeven analysis for profitable routes.")
else:
    print("Most profitable routes data is not available. Skipping breakeven calculation for all profitable routes.")


--- Calculating Breakeven Analysis for All Profitable Routes ---

--- Calculating Breakeven Flights (Airplane Cost: $90,000,000) ---
  CANONICAL_ROUTE_PAIR  PROFIT_PER_FLIGHT  BREAKEVEN_ROUND_TRIP_FLIGHTS  \
0              JFK-LAX           31518.67                       2856.00   
1              DCA-ORD           39804.35                       2262.00   
2              DCA-LGA           41194.65                       2185.00   
3              BOS-LGA           27574.06                       3264.00   
4              ATL-CLT           42876.29                       2100.00   
5              LAX-SFO           15279.45                       5891.00   
6              CLT-GSP           80042.03                       1125.00   
7              JFK-SFO           31197.75                       2885.00   
8              DFW-IAH           38779.52                       2321.00   
9              LGA-ORD           15792.57                       5699.00   

   TOTAL_PROFIT  ROUND_TRIP_FLIGHTS  AVG

### Task 2.2: Route Recommendation (Q3 - Using Advanced Scoring)
 - Define recommendation criteria (as per your detailed approach).
 - Prepare a comprehensive DataFrame for scoring.
 - Implement advanced scoring logic to select 5 routes.
 - Justify the selection of 5 routes in detail.

 **Route Recommendation Criteria & Scoring Model:**
 (This markdown section should summarize your detailed approach: Key Considerations, Model Design, Metrics & Importance, Weight Distribution, as you outlined)

 Example Summary:
 The recommendation model uses a multi-factored approach:
 1. **Metrics:** Total Profit, Profit Per Flight, Flight Volume (Round Trip Flights), Average Occupancy, Average Total Delay (Departure + Arrival), and Breakeven Round Trip Flights.
 2. **Normalization:** Min-Max scaling (0-100) for each metric.
 3. **Weights:**
     - Profitability (Total Profit 25%, Profit Per Flight 15%) = 40%
     - Market Demand (Flight Volume) = 20%
     - Operational Factors (Punctuality/Avg Total Delay 15%, Occupancy 10%) = 25%
     - Investment Efficiency (Breakeven Flights Score) = 15%
 4. **Scoring:** Weighted sum of normalized scores.

In [15]:
# Prepare data for advanced recommendation
all_metrics_for_scoring_df = pd.DataFrame()
advanced_recommendations_df = pd.DataFrame()

if 'most_profitable_routes_df' in locals() and not most_profitable_routes_df.empty:
    all_metrics_for_scoring_df = most_profitable_routes_df.copy()

    # Merge breakeven analysis data (BREAKEVEN_ROUND_TRIP_FLIGHTS)
    if 'breakeven_for_all_profitable_df' in locals() and not breakeven_for_all_profitable_df.empty:
        all_metrics_for_scoring_df = pd.merge(
            all_metrics_for_scoring_df,
            breakeven_for_all_profitable_df[['CANONICAL_ROUTE_PAIR', 'BREAKEVEN_ROUND_TRIP_FLIGHTS']],
            on='CANONICAL_ROUTE_PAIR',
            how='left' 
            # Left join is appropriate: we want to score all profitable routes, 
            # and add breakeven info if available (it should be for those with positive profit per flight)
        )
    else:
        print("Warning: Breakeven data for all profitable routes not available. 'BREAKEVEN_ROUND_TRIP_FLIGHTS' will be missing for scoring.")
        all_metrics_for_scoring_df['BREAKEVEN_ROUND_TRIP_FLIGHTS'] = np.nan # Ensure column exists if expected by scorer

    # The 'ROUND_TRIP_FLIGHTS' for volume is already in most_profitable_routes_df.
    # If busiest_routes_df had a more definitive or different volume metric, it could be merged here.
    # For now, we assume most_profitable_routes_df.ROUND_TRIP_FLIGHTS is the primary volume indicator for these routes.
    
    print(f"\n--- Q3: Recommending 5 Round Trip Routes for Investment (Advanced Scoring) ---")
    print(f"Shape of data input to scoring: {all_metrics_for_scoring_df.shape}")
    if not all_metrics_for_scoring_df.empty:
        advanced_recommendations_df = analysis_utils.recommend_routes_advanced_scoring(
            all_metrics_for_scoring_df,
            num_recommendations=5
        )

        if not advanced_recommendations_df.empty:
            print("\n--- Top 5 Recommended Routes (Advanced Scoring) ---")
            # Display relevant columns for the recommendation justification
            cols_to_display_rec = ['RANK', 'CANONICAL_ROUTE_PAIR', 'COMPOSITE_SCORE',
                                   'TOTAL_PROFIT', 'PROFIT_PER_FLIGHT', 'ROUND_TRIP_FLIGHTS',
                                   'AVG_TOTAL_DELAY', 'AVG_OCCUPANCY', 'BREAKEVEN_ROUND_TRIP_FLIGHTS']
            # Add normalized scores if you want to see them
            norm_cols_to_add = [col for col in advanced_recommendations_df.columns if col.startswith('NORM_')]
            cols_to_display_rec.extend(norm_cols_to_add)
            
            cols_to_display_rec = [col for col in cols_to_display_rec if col in advanced_recommendations_df.columns]
            print(advanced_recommendations_df[cols_to_display_rec])
            
            # Save intermediate output
            step_name = "07_advanced_recommendations"
            file_name = f"{step_name}.csv"; output_file_path = os.path.join(config.STEP_OUTPUT_DIR, file_name)
            df_to_save = advanced_recommendations_df.copy()
            df_to_save.loc[:, config.BATCH_ID_COLUMN] = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
            df_to_save.to_csv(output_file_path, index=False); print(f"Saved DataFrame to: {output_file_path}")
        else:
            print("Could not make route recommendations using advanced scoring.")
    else:
        print("Data for scoring (all_metrics_for_scoring_df) is empty. Skipping advanced recommendation.")
else:
    print("Most profitable routes data is not available. Cannot proceed with advanced recommendations.")


--- Q3: Recommending 5 Round Trip Routes for Investment (Advanced Scoring) ---
Shape of data input to scoring: (10, 12)

--- Recommend 5 Routes (Advanced Scoring) ---
Composite Score calculated using: (NORM_TOTAL_PROFIT * 0.25) + (NORM_PROFIT_PER_FLIGHT * 0.15) + (NORM_ROUND_TRIP_FLIGHTS * 0.2) + (NORM_AVG_TOTAL_DELAY * 0.15) + (NORM_AVG_OCCUPANCY * 0.1) + (NORM_BREAKEVEN_ROUND_TRIP_FLIGHTS * 0.15)

--- Top 5 Recommended Routes (Advanced Scoring) ---
   RANK CANONICAL_ROUTE_PAIR  COMPOSITE_SCORE  TOTAL_PROFIT  \
0     1              JFK-LAX            72.02   99252298.23   
2     2              DCA-LGA            50.09   69124622.46   
6     3              CLT-GSP            46.59   61872487.00   
3     4              BOS-LGA            46.14   66398343.46   
1     5              DCA-ORD            45.28   73399227.77   

   PROFIT_PER_FLIGHT  ROUND_TRIP_FLIGHTS  AVG_OCCUPANCY  AVG_TOTAL_DELAY  \
0           31518.67                3149           0.65           -11.78   
2           4

 ### Task 2.3: Breakeven Analysis for Recommended Routes (Q4 - Display)
 - Display breakeven points and key summary components for the 5 routes *that were just recommended* by the advanced scoring model.

In [16]:
# Display breakeven details specifically for the 5 recommended routes
if 'advanced_recommendations_df' in locals() and not advanced_recommendations_df.empty and \
   'breakeven_for_all_profitable_df' in locals() and not breakeven_for_all_profitable_df.empty:

    print("\n--- Q4: Breakeven Analysis for the 5 Recommended Routes ---")
    
    # Filter the comprehensive breakeven data to only the recommended routes
    recommended_route_pairs = advanced_recommendations_df['CANONICAL_ROUTE_PAIR'].tolist()
    final_breakeven_display_df = breakeven_for_all_profitable_df[
        breakeven_for_all_profitable_df['CANONICAL_ROUTE_PAIR'].isin(recommended_route_pairs)
    ].copy()

    # Merge with rank from recommendations for ordered display
    final_breakeven_display_df = pd.merge(
        final_breakeven_display_df,
        advanced_recommendations_df[['CANONICAL_ROUTE_PAIR', 'RANK', 'COMPOSITE_SCORE']],
        on='CANONICAL_ROUTE_PAIR',
        how='left'
    ).sort_values(by='RANK')


    if not final_breakeven_display_df.empty:
        print(f"\nBreakeven Analysis (Airplane Upfront Cost: ${config.AIRPLANE_UPFRONT_COST:,.0f} per route) for Recommended Routes:")
        cols_to_display_final_be = ['RANK', 'CANONICAL_ROUTE_PAIR', 'PROFIT_PER_FLIGHT', 
                                    'BREAKEVEN_ROUND_TRIP_FLIGHTS', 'TOTAL_PROFIT', 
                                    'ROUND_TRIP_FLIGHTS', 'AVG_DEP_DELAY', 'AVG_ARR_DELAY', 'COMPOSITE_SCORE']
        cols_to_display_final_be = [col for col in cols_to_display_final_be if col in final_breakeven_display_df.columns]
        print(final_breakeven_display_df[cols_to_display_final_be])
        # Save intermediate output
        step_name = "08_breakeven_for_recommended_routes"
        file_name = f"{step_name}.csv"; output_file_path = os.path.join(config.STEP_OUTPUT_DIR, file_name)
        df_to_save = final_breakeven_display_df.copy()
        df_to_save.loc[:, config.BATCH_ID_COLUMN] = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
        df_to_save.to_csv(output_file_path, index=False); print(f"Saved DataFrame to: {output_file_path}")

    else:
        print("Could not display breakeven analysis for recommended routes (e.g., if no routes were recommended or breakeven data missing).")

elif 'advanced_recommendations_df' in locals() and advanced_recommendations_df.empty:
    print("No routes were recommended by the advanced scoring model. Skipping final breakeven display.")
else:
    print("Prerequisite data for displaying breakeven of recommended routes is missing. Skipping.")

print("\n--- Final Data Quality Log Summary ---")
if 'dq_utils' in globals() and hasattr(dq_utils, 'display_dq_summary'):
    dq_utils.display_dq_summary()
else:
    print("dq_utils not available to display final summary.")



--- Q4: Breakeven Analysis for the 5 Recommended Routes ---

Breakeven Analysis (Airplane Upfront Cost: $90,000,000 per route) for Recommended Routes:
   RANK CANONICAL_ROUTE_PAIR  PROFIT_PER_FLIGHT  BREAKEVEN_ROUND_TRIP_FLIGHTS  \
0     1              JFK-LAX           31518.67                       2856.00   
2     2              DCA-LGA           41194.65                       2185.00   
4     3              CLT-GSP           80042.03                       1125.00   
3     4              BOS-LGA           27574.06                       3264.00   
1     5              DCA-ORD           39804.35                       2262.00   

   TOTAL_PROFIT  ROUND_TRIP_FLIGHTS  AVG_DEP_DELAY  AVG_ARR_DELAY  \
0   99252298.23                3149          -1.51         -10.27   
2   69124622.46                1678          -2.72          -5.26   
4   61872487.00                 773          -1.28          -6.71   
3   66398343.46                2408          -2.49          -8.00   
1   73399227.77 

In [17]:
print("\n--- Final Data Quality Log Summary ---")
if 'dq_utils' in globals() and hasattr(dq_utils, 'display_dq_summary'):
    dq_utils.display_dq_summary()
else:
    print("dq_utils not available to display final summary.")


--- Final Data Quality Log Summary ---

--- Data Quality Log Summary ---
Run ID: 49ec5f94-94e7-4896-94ba-67fc53ec3155
Total DQ metrics logged: 79
Last 5 entries:
              Timestamp                                 RunID     Phase  \
74  2025-05-13 21:41:12  49ec5f94-94e7-4896-94ba-67fc53ec3155  Analysis   
75  2025-05-13 21:41:12  49ec5f94-94e7-4896-94ba-67fc53ec3155  Analysis   
76  2025-05-13 21:41:12  49ec5f94-94e7-4896-94ba-67fc53ec3155  Analysis   
77  2025-05-13 21:41:12  49ec5f94-94e7-4896-94ba-67fc53ec3155  Analysis   
78  2025-05-13 21:41:12  49ec5f94-94e7-4896-94ba-67fc53ec3155  Analysis   

                                     Step    TableName  \
74  Recommend 5 Routes (Advanced Scoring)  AdvancedRec   
75  Recommend 5 Routes (Advanced Scoring)  AdvancedRec   
76  Recommend 5 Routes (Advanced Scoring)  AdvancedRec   
77  Recommend 5 Routes (Advanced Scoring)  AdvancedRec   
78  Recommend 5 Routes (Advanced Scoring)  AdvancedRec   

                          Metric     

## Phase 3: Reporting & Strategic Outlook

### Task 3.1: Key Performance Indicators (KPIs) - Simplified Core Set for Initial Launch

**Listed in Tableau Presentation**

### Task 3.2: "What's Next?" - Future Considerations
 - Outline future work, potential model enhancements, or further data product development ideas.
 - Include thoughts on enhancing the DQ monitoring framework.

 **"What's Next?" - Future Considerations & Enhancements:**

 While this analysis provides a strong foundation, several areas could be explored for future enhancements to deliver even better insights and a more robust data product:

 1.  **Identify and Explore AI/ML Models for Better Prediction of Best Launch Routes**
     * **Idea:** Investigate and prototype machine learning models (e.g., regression models to predict route profitability/demand, or Learning to Rank models if sufficient historical/proxy data could be acquired) using the existing and expanded feature set. This would move beyond the current weighted scoring.
     * **Benefit:** Potentially uncovers more complex patterns and interactions between route characteristics that lead to success, leading to more data-driven and nuanced route selection. This could refine or challenge initial recommendations and improve the prediction of viability for entirely new, unproven routes.

 2.  **Conduct In-depth Seasonality Analysis:**
     * **Idea:** The current analysis disregards seasonal effects as per initial instructions. A crucial next step would be to acquire or model data spanning multiple quarters/years to analyze seasonal trends in demand, fare elasticity, and operational performance (e.g., weather-related delays) for key routes.
     * **Benefit:** Allows for more accurate demand forecasting, optimized pricing strategies that adapt to peak and off-peak seasons, and better resource allocation (e.g., scheduling maintenance during lower demand periods). This directly improves the accuracy of profitability projections.

 3.  **Develop Dynamic Demand & Pricing Models:**
     * **Idea:** Move beyond the static 1Q2019 analysis by incorporating time-series forecasting for passenger demand on promising routes. Develop models to understand price elasticity and simulate the impact of different fare structures.
     * **Benefit:** Enables more proactive revenue management, optimized pricing strategies considering seasonality (see next point) and booking patterns, and better capacity planning.
     * **Fleet Optimization:** If different aircraft types are considered, model which aircraft is best suited for which route based on capacity, range, operating economics, and demand.

 4.  **Customer-Centric Analytics:**
     * **Idea:** Analyze passenger data (demographics, travel patterns, booking behavior) to identify distinct customer segments.
     * **Benefit:** Moves beyond route-level profit to customer lifetime value, informing marketing, loyalty programs, and service improvements that truly resonate with valuable customer segments.

 5.  **Improving the Data Quality (DQ) Monitoring Framework:**
     * **Automated DQ Alerts & Anomaly Detection:** Implement systems that automatically flag significant DQ issues or anomalies in real-time or near real-time (e.g., sudden drop in data completeness for a key source, unusual spike in reported delays, fare data outside expected ranges).
     * **DQ Root Cause Analysis Tools:** Enhance the DQ dashboard in Tableau (or other BI tools) with drill-down capabilities to more easily trace the origin of data quality problems.
     * **Data Lineage Visualization:** Implement tools to visualize data lineage, showing how data flows from source systems through transformations to the final analytical tables and reports. This aids in understanding impact and tracing errors.
     * **DQ Impact Quantification:** Develop methodologies to estimate the potential business impact of identified data quality issues (e.g., "A 5% error in occupancy data could lead to an X% misstatement in projected revenue for route Y").
     * **Proactive DQ Rules & Validation:** Embed more complex validation rules directly into data ingestion and processing pipelines (e.g., using tools like Great Expectations) to catch issues earlier.
     * **Feedback Loop for DQ Issues:** Formalize the process for data consumers to report DQ issues and for these issues to be tracked, prioritized, and resolved by data engineering or source system owners.

 These future steps would transform the current analytical product into a more dynamic, predictive, and strategically invaluable asset for the airline, supporting a wider range of operational and planning decisions.

 ### Task 3.3: Documentation & Submission Packaging
 - Ensure all Python scripts (`airline_scripts/*.py`) are well-commented with docstrings for functions and explanations for complex logic. (This is an ongoing task within the `.py` files themselves).
 - Ensure this Jupyter Notebook (`Airline_Route_Analysis.ipynb`) has clear markdown explanations for each step, interprets results, documents assumptions, and presents the overall narrative. (This is achieved through the markdown cells throughout this notebook).
 - Create/update `README.md` with a project overview, setup instructions (referencing `requirements.txt`), and guidance on how to run the notebook and view Tableau dashboards. (This is an external file).
 - Generate/finalize `requirements.txt` listing all Python dependencies. (This can be done using `pip freeze > requirements.txt` in the project's virtual environment).
 - Package all deliverables (notebook, scripts, reports, logs, README, requirements.txt, Tableau workbook) into a single ZIP file for submission. (This is the final external packaging step).

 ---
 ## End of Analysis